In [1]:
import dash
from dash import html 
from dash import dcc
from dash.dependencies import Input, Output
from datetime import datetime, timedelta, date
from meteostat import Point, Daily

import plotly.express as px
import dash_leaflet as dl
from jupyter_dash import JupyterDash

In [2]:
#print the versions of the libraries
print(dash.__version__)
print(html.__version__)

import meteostat
print(meteostat.__version__)

import plotly
print(f'Plotly version: {plotly.__version__}')
print(dl.__version__)

import jupyter_dash
print(jupyter_dash.__version__)

2.7.0
2.0.6
1.6.5
Plotly version: 5.11.0
0.1.23
0.4.2


In [28]:
from meteostat import Point, Daily
from datetime import datetime

import plotly.express as px

#KOS

# Set time period
start = datetime(2021, 1, 1)
end = datetime(2021, 12, 31)

# Create Point
location = Point(36.8925871 , 27.2877926, 70) #lat, long, altitude

# Get daily data for 2018
data = Daily(location, start, end)
data = data.fetch()

print(data)

            tavg  tmin  tmax  prcp  snow   wdir  wspd  wpgt    pres  tsun
time                                                                     
2021-01-01  15.6  13.7  17.9   NaN   NaN   97.0   7.7  22.2  1018.1   NaN
2021-01-02  16.2  13.9  19.8   NaN   NaN  101.0   6.0  22.2  1021.4   NaN
2021-01-03  15.5  13.5  19.2   NaN   NaN   33.0   3.6  14.8  1021.9   NaN
2021-01-04  16.3  12.9  19.0   NaN   NaN  100.0  11.4  38.9  1017.7   NaN
2021-01-05  16.7  13.4  19.3   NaN   NaN  162.0  10.2  44.5  1017.1   NaN
...          ...   ...   ...   ...   ...    ...   ...   ...     ...   ...
2021-12-27  15.7  13.8  19.0   NaN   NaN  117.0   7.4  20.4  1020.9   NaN
2021-12-28  15.8  14.1  18.4   NaN   NaN  116.0  10.7  25.9  1016.0   NaN
2021-12-29  14.6  13.1  15.6   NaN   NaN  124.0  14.9  33.3  1010.5   NaN
2021-12-30  13.2  10.9  14.9   NaN   NaN   64.0   7.5  24.1  1005.8   NaN
2021-12-31  15.2  12.6  20.5   NaN   NaN   31.0   5.5  16.7  1012.8   NaN

[365 rows x 10 columns]


In [16]:
# from datetime import date, timedelta

# today = date.today()
# five_days_ago = date.today() - timedelta(days=5)
# print(five_days_ago)

In [60]:
lat = 35.0
lon = 25.0
start = datetime(2021, 1, 1)
end = datetime(2021, 12, 31)


location = Point(lat, lon, 70)

data = Daily(location, start, end)
data = data.fetch()

data = data.reset_index()

print("\nData columns:\n", data.columns)


Data columns:
 Index(['time', 'tavg', 'tmin', 'tmax', 'prcp', 'snow', 'wdir', 'wspd', 'wpgt',
       'pres', 'tsun'],
      dtype='object')


In [61]:
# Rename the columns
print(data.rename(columns={'tmin': 'Tmin',
                           'tmax': 'Tmax'}))	

          time  tavg  Tmin  Tmax  prcp  snow   wdir  wspd  wpgt    pres  tsun
0   2021-01-01  13.6   9.5  18.4   NaN   NaN  108.0   6.9   NaN  1016.6   NaN
1   2021-01-02  14.6  10.0  18.8   NaN   NaN  111.0   8.0   NaN  1018.8   NaN
2   2021-01-03  14.7  11.0  19.3   NaN   NaN  116.0   6.9   NaN  1019.2   NaN
3   2021-01-04  15.4  12.5  18.8   NaN   NaN  146.0  12.1   NaN  1014.8   NaN
4   2021-01-05  13.4   8.6  17.6   NaN   NaN  316.0   9.6   NaN  1017.0   NaN
..         ...   ...   ...   ...   ...   ...    ...   ...   ...     ...   ...
360 2021-12-27  13.0   8.9  17.5   NaN   NaN  152.0   8.3   NaN  1019.3   NaN
361 2021-12-28  13.6  10.8  17.0   NaN   NaN  165.0  11.6   NaN  1013.6   NaN
362 2021-12-29  13.7  11.7  16.4   NaN   NaN  158.0   8.9   NaN  1007.2   NaN
363 2021-12-30  12.6  11.1  15.1   NaN   NaN   17.0   8.8   NaN  1003.5   NaN
364 2021-12-31  14.3  11.7  16.4   NaN   NaN   13.0  15.2   NaN  1008.9   NaN

[365 rows x 11 columns]


## DASH

In [14]:
# import dash
# from dash import html #import dash_core_components as dcc
# from dash import dcc
# from dash.dependencies import Input, Output
# from datetime import datetime, timedelta, date
# from meteostat import Point, Daily

# import plotly.express as px
# import dash_leaflet as dl
# from jupyter_dash import JupyterDash

app = JupyterDash(__name__, external_stylesheets=['https://maxcdn.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css'])

def plot_tmax_boxplot(lat, lon, start, end, variable):
    location = Point(lat, lon, 70)
    
    data = Daily(location, start, end)
    data = data.fetch()

    data=data.reset_index()

    data['month'] = data['time'].dt.month

    fig = px.box(data, x='Month', y=variable, title='Monthly Tmax Boxplot')
    return fig

def plot_weather_data(lat, lon, start, end):
    location = Point(lat, lon, 70)
    
    data = Daily(location, start, end)
    data = data.fetch()

    data=data.reset_index()

    fig = px.line(data, x='time', y=['tmin','tmax'],
                  title=f'Temperature min/max for the Given Location ({lat:.2f},{lon:.2f})',
                  labels={'time': 'Date', 'tmin': 'Min Temp', 'tmax': 'Max Temp'})

    fig.update_yaxes(title_text='Temperature')


    return (fig)

app.layout = html.Div([
    html.H1('Trip Planner', style={'textAlign': 'center', 'padding': '20px'}),
    dcc.DatePickerRange(
        id='date-picker',
        min_date_allowed=datetime(2000, 1, 1),
        max_date_allowed= date.today() - timedelta(days=7), #datetime(2022, 12, 31),
        start_date=datetime(2021, 1, 1),
        end_date=datetime(2022, 12, 31),
        display_format='MMM DD, YYYY'
    ),
    dl.Map(
        [dl.TileLayer(), dl.LayerGroup(id="layer")],
        id='map',
        style={'width': '100%', 'height': '50vh', 'margin': "auto", "display": "block"},
        center=[35, 25],
        zoom=4,
        #click_lat_lng=True,
    ),
    dcc.Graph(id='weather-plot'),
    html.Label('Select variable for boxplot:'),
    dcc.Dropdown(
        id='boxplot-variable',
        options=[
            {'label': 'Tmax', 'value': 'tmax'},
            {'label': 'Tmin', 'value': 'tmin'}
        ],
        value='tmax'
    ),
    dcc.Graph(id='tmax-boxplot')
])

@app.callback(
    Output('layer', 'children'),
    [Input('map', 'click_lat_lng')],
)
def update_markers(click_lat_lng):
    if not click_lat_lng:
        click_lat_lng = [35, 25]
    return [dl.Marker(position=click_lat_lng, children=dl.Tooltip(f"({click_lat_lng[0]:.2f}, {click_lat_lng[1]:.2f})"))]

@app.callback(
    [Output('weather-plot', 'figure'), Output('tmax-boxplot', 'figure')],
    [Input('map', 'click_lat_lng'), Input('date-picker', 'start_date'), Input('date-picker', 'end_date'), Input('boxplot-variable', 'value')],
)
def update_weather_plots(click_lat_lng, start_date, end_date, boxplot_variable):
    if not click_lat_lng:
        lat, lon = 35, 25
    else:
        lat = int(click_lat_lng[0])
        lon = int(click_lat_lng[1])
    
   
    start = datetime(int(start_date[0:4]),
                    int(start_date[5:7]),
                    int(start_date[8:10]))
    end = datetime(2022, 12, 31)
    
    line_plot = plot_weather_data(lat, lon, start, end)
    
    
    box_plot = plot_tmax_boxplot(lat, lon, start, end, boxplot_variable)
    
    return line_plot, box_plot
    

# Start of the application
if __name__ == '__main__':
    app.run_server(mode="inline", port=8054)


In [64]:
# import dash
# from dash import html #import dash_core_components as dcc
# from dash import dcc
# from dash.dependencies import Input, Output
# from datetime import datetime, timedelta, date
# from meteostat import Point, Daily

# import plotly.express as px
# import dash_leaflet as dl
# from jupyter_dash import JupyterDash

app = JupyterDash(__name__, external_stylesheets=['https://maxcdn.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css'])

def plot_tmax_boxplot(lat, lon, start, end, variable):
    location = Point(lat, lon, 70)
    
    data = Daily(location, start, end)
    data = data.fetch()

    data = data.reset_index()

    data['month'] = data['time'].dt.month_name()

    fig = px.box(data, x='month', y=variable, title='Monthly Tmax Boxplot')

    fig.update_xaxes(title_text='Month')
    fig.update_yaxes(title_text='Tmax' if variable == 'tmax' else 'Tmin')
    return fig

def plot_weather_data(lat, lon, start, end):
    location = Point(lat, lon, 70)
    
    data = Daily(location, start, end)
    data = data.fetch()

    data = data.reset_index()

    # Rename the columns
    data = data.rename(columns={'tmin': 'Tmin',
                           'tmax': 'Tmax'})

    fig = px.line(data, x='time', y=['Tmin', 'Tmax'],
                  title=f'Temperature Min/Max for the Given Location ({lat:.2f}, {lon:.2f})',
                  labels={'time': 'Date', 'value': 'Temperature'})
    fig.update_yaxes(title_text='Temperature')

    return fig


app.layout = html.Div([
    html.H1('Trip Planner', style={'textAlign': 'center', 'padding': '20px'}),
    dcc.DatePickerRange(
        id='date-picker',
        min_date_allowed=datetime(2000, 1, 1),
        max_date_allowed= date.today() - timedelta(days=7),
        start_date=datetime(2021, 1, 1),
        end_date=datetime(2022, 12, 31),
        display_format='MMM DD, YYYY',
        style = {
                        'font-size': '6px','display': 'inline-block', 'border-radius' : '2px', 
                        'border' : '1px solid #ccc', 'color': '#333', 
                        'border-spacing' : '0', 'border-collapse' :'separate'
                        } 
    ),
    dl.Map(
        [dl.TileLayer(), dl.LayerGroup(id="layer")],
        id='map',
        style={'width': '100%', 'height': '50vh', 'margin': "auto", "display": "block"},
        center=[35, 25],
        zoom=4,
        #click_lat_lng=True,
    ),
    dcc.Graph(id='weather-plot'),
    html.Label('Select variable for boxplot:'),
    dcc.Dropdown(
        id='boxplot-variable',
        options=[
            {'label': 'Tmax', 'value': 'tmax'},
            {'label': 'Tmin', 'value': 'tmin'}
        ],
        value='tmax'
    ),
    dcc.Graph(id='tmax-boxplot'),
    html.Div([
        html.H4("About"),
        html.A("My Blog - FossEngineer", href="https://fossengineer.com", target="_blank"),
        html.Br(),
        html.A("About this App - FossEngineer", href="https://fossengineer.com/python-trip-planner/", target="_blank"),
        html.Br(),
        html.A("Source Code", href="https://github.com/JAlcocerT/Py_Trip_Planner/", target="_blank"),
    ], style={'float': 'right'})
])

@app.callback(
    Output('layer', 'children'),
    [Input('map', 'click_lat_lng')],
)
def update_markers(click_lat_lng):
    if not click_lat_lng:
        click_lat_lng = [35, 25]
    return [dl.Marker(position=click_lat_lng, children=dl.Tooltip(f"({click_lat_lng[0]:.2f}, {click_lat_lng[1]:.2f})"))]

@app.callback(
    [Output('weather-plot', 'figure'), Output('tmax-boxplot', 'figure')],
    [Input('map', 'click_lat_lng'), Input('date-picker', 'start_date'), Input('date-picker', 'end_date'), Input('boxplot-variable', 'value')],
)
def update_weather_plots(click_lat_lng, start_date, end_date, boxplot_variable):
    if not click_lat_lng:
        lat, lon = 35, 25
    else:
        lat = int(click_lat_lng[0])
        lon = int(click_lat_lng[1])
    
   
    start = datetime(int(start_date[0:4]),
                    int(start_date[5:7]),
                    int(start_date[8:10]))
    end = datetime(2022, 12, 31)
    
    line_plot = plot_weather_data(lat, lon, start, end)
    
    
    box_plot = plot_tmax_boxplot(lat, lon, start, end, boxplot_variable)
    
    return line_plot, box_plot
    

# Start of the application
if __name__ == '__main__':
    app.run_server(mode="inline", port=8054)

In [3]:
import dash
from dash import html #import dash_core_components as dcc
from dash import dcc
from dash.dependencies import Input, Output
from datetime import datetime, timedelta, date
from meteostat import Point, Daily

import plotly.express as px
import dash_leaflet as dl
from jupyter_dash import JupyterDash

app = JupyterDash(__name__, external_stylesheets=['https://maxcdn.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css'])

def plot_tmax_boxplot(lat, lon, start, end, variable):
    location = Point(lat, lon, 70)
    
    data = Daily(location, start, end)
    data = data.fetch()

    data = data.reset_index()

    data['month'] = data['time'].dt.month_name()

    fig = px.box(data, x='month', y=variable, title='Monthly Tmax Boxplot')

    fig.update_xaxes(title_text='Month')
    fig.update_yaxes(title_text='Tmax' if variable == 'tmax' else 'Tmin')
    return fig

def plot_weather_data(lat, lon, start, end):
    location = Point(lat, lon, 70)
    
    data = Daily(location, start, end)
    data = data.fetch()

    data = data.reset_index()

    # Rename the columns
    data = data.rename(columns={'tmin': 'Tmin',
                           'tmax': 'Tmax'})

    fig = px.line(data, x='time', y=['Tmin', 'Tmax'],
                  title=f'Temperature Min/Max for the Given Location ({lat:.2f}, {lon:.2f})',
                  labels={'time': 'Date', 'value': 'Temperature'})
    fig.update_yaxes(title_text='Temperature')

    return fig


app.layout = html.Div([
    html.H1('Trip Planner', style={'textAlign': 'center', 'padding': '20px'}),
    dcc.DatePickerRange(
        id='date-picker',
        min_date_allowed=datetime(2000, 1, 1),
        max_date_allowed= date.today() - timedelta(days=7),
        start_date=datetime(2021, 1, 1),
        end_date=datetime(2022, 12, 31),
        display_format='MMM DD, YYYY',
        style = {
                        'font-size': '6px','display': 'inline-block', 'border-radius' : '2px', 
                        'border' : '1px solid #ccc', 'color': '#333', 
                        'border-spacing' : '0', 'border-collapse' :'separate'
                        } 
    ),
    dl.Map(
        [dl.TileLayer(), dl.LayerGroup(id="layer")],
        id='map',
        style={'width': '100%', 'height': '50vh', 'margin': "auto", "display": "block"},
        center=[35, 25],
        zoom=4,
        #click_lat_lng=True,
    ),
    dcc.Graph(id='weather-plot'),
    html.Label('Select variable for boxplot:'),
    dcc.Dropdown(
        id='boxplot-variable',
        options=[
            {'label': 'Tmax', 'value': 'tmax'},
            {'label': 'Tmin', 'value': 'tmin'}
        ],
        value='tmax'
    ),
    dcc.Graph(id='tmax-boxplot'),
    html.Div([
        html.H4("About"),
        html.A("My Blog - FossEngineer", href="https://fossengineer.com", target="_blank"),
        html.Br(),
        html.A("About this App - FossEngineer", href="https://fossengineer.com/python-trip-planner/", target="_blank"),
        html.Br(),
        html.A("Source Code", href="https://github.com/JAlcocerT/Py_Trip_Planner/", target="_blank"),
    ], style={'float': 'right'})
])

@app.callback(
    Output('layer', 'children'),
    [Input('map', 'click_lat_lng')],
)
def update_markers(click_lat_lng):
    if not click_lat_lng:
        click_lat_lng = [35, 25]
    return [dl.Marker(position=click_lat_lng, children=dl.Tooltip(f"({click_lat_lng[0]:.2f}, {click_lat_lng[1]:.2f})"))]

@app.callback(
    [Output('weather-plot', 'figure'), Output('tmax-boxplot', 'figure')],
    [Input('map', 'click_lat_lng'), Input('date-picker', 'start_date'), Input('date-picker', 'end_date'), Input('boxplot-variable', 'value')],
)
def update_weather_plots(click_lat_lng, start_date, end_date, boxplot_variable):
    if not click_lat_lng:
        lat, lon = 35, 25
    else:
        lat = int(click_lat_lng[0])
        lon = int(click_lat_lng[1])
    
   
    start = datetime(int(start_date[0:4]),
                    int(start_date[5:7]),
                    int(start_date[8:10]))
    end = datetime(2022, 12, 31)
    
    line_plot = plot_weather_data(lat, lon, start, end)
    
    
    box_plot = plot_tmax_boxplot(lat, lon, start, end, boxplot_variable)
    
    return line_plot, box_plot
    

# Start of the application
if __name__ == '__main__':
    app.run_server(mode="inline", port=8054)